In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [8]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [10]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [11]:
ohe_encoder_geography=OneHotEncoder(handle_unknown='ignore')
geo_encoded=ohe_encoder_geography.fit_transform(df[['Geography']]).toarray()
ohe_encoded_df=pd.DataFrame(geo_encoded,columns=ohe_encoder_geography.get_feature_names_out(['Geography']))
ohe_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [12]:
df=pd.concat([df.drop(['Geography'],axis=1),ohe_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


### Independant and Dependant Features

In [13]:
X=df.drop('EstimatedSalary',axis=1)
y=df['EstimatedSalary']

### Train-Test Split

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [15]:
# Feature Scaling
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('ohe_encoder_geography.pkl','wb') as file:
    pickle.dump(ohe_encoder_geography,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN-Regression Problem Statement

In [22]:
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [18]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # HL1 Connected with input layer
    Dense(32,activation='relu'), # HL2
    Dense(1) # Output Layer for regression
])

# Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Setup Tensorboard
log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [24]:
# Setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [25]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    )

Epoch 1/100


250/250 [==============================] - 4s 6ms/step - loss: 100353.1406 - mae: 100353.1406 - val_loss: 98499.3672 - val_mae: 98499.3672
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 99402.6719 - mae: 99402.6719 - val_loss: 96584.8984 - val_mae: 96584.8984
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 96093.5391 - mae: 96093.5391 - val_loss: 91781.9297 - val_mae: 91781.9297
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 89727.2734 - mae: 89727.2734 - val_loss: 84032.2656 - val_mae: 84032.2656
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 80738.5469 - mae: 80738.5469 - val_loss: 74366.9922 - val_mae: 74366.9922
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 70886.8125 - mae: 70886.8125 - val_loss: 64871.6055 - val_mae: 64871.6055
Epoch 7/100
250/250 [==============================] - 2s 8ms/step - loss: 62159.9805 - mae: 62159.980

In [26]:
# Load Tensorboard Extension
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit

In [28]:
model.save('regression_model.h5')

d:\Projects\MachineLearningPractice\DeepLearning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
